> First time use: follow instructions in the README.md file in this directory.


**[PT]** Português

---

**[EN]** English


# Naturalidade


Análise dos valores da informação sobre a naturalidade dos estudantes

---

# Place of Birth

Information related to place of birth of students


## Setup

In [1]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)

## Lista de lugares diferentes e número de ocorrências

---

## List of different places with number of occurrences

In [2]:
from timelinknb.pandas import attribute_values

naturalidades = attribute_values('naturalidade',dates_between=('1500-00-00','1990-00-00'))
naturalidades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11499 entries, Lisboa to Óvoa, Viseu
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   count     11499 non-null  int64 
 1   date_in   11499 non-null  object
 2   date_max  11499 non-null  object
dtypes: int64(1), object(2)
memory usage: 359.3+ KB


### Lugares principais

---

### Main locations

In [3]:
naturalidades.sort_values('count', ascending=False).head(20)


,count,date_in,date_max
value,,,
Lisboa,8784,1537-02-12,1916-07-19
Coimbra,5526,1537-00-00,1915-10-12
Porto,3391,1537-05-30,1917-10-22
Braga,1608,1540-01-21,1914-07-24
Évora,1072,1537-11-22,1910-10-10
Viseu,986,1537-00-00,1912-07-03
Guimarães,980,1537-12-18,1912-07-18
Lamego,972,1537-00-00,1909-10-05
Aveiro,790,1538-04-21,1913-10-13


### Lugares com menos de 5 estudantes

---

### Locations with less than five students

In [4]:
naturalidades[naturalidades['count']<5].sort_values('count', ascending=False).head(20)

,count,date_in,date_max
value,,,
Abrigada,4,1646-10-22,1738-11-04
Paio Mendes,4,1686-10-01,1696-10-01
Santa Eulália de Barrosas,4,1842-10-26,1910-11-16
Santa Eulália da Palmeira,4,1704-10-01,1725-10-01
Santa Eugénia,4,1743-10-01,1787-10-31
"Santa Comba, Seia",4,1746-10-01,1913-07-16
Sanfins do Douro,4,1597-11-22,1891-10-31
"Sande, Lamego",4,1684-10-01,1764-10-01
Samora Correia,4,1698-10-01,1783-10-31


## Identificação de topónimos

---

## Geocoding

https://craftingdh.netlify.app/tutorials/folium/

In [5]:
!pip install geopy

In [6]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="fauc1530-1919", timeout=2)

#### Testes

---

#### Testing

In [7]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

geolocator = Nominatim(user_agent="fauc1530-1919", timeout=2)
# abide to https://operations.osmfoundation.org/policies/nominatim/
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 
plp = ['ao','mz','cv','st','mo']
other_countries = ['es','it','fr']

Colocar o topónimo a localizar

---

Set the place to look for

In [8]:

place="Porto"



In [9]:
import pandas as pd

places_cache_cols = ['naturalidade','geocoder','id','address','city','country','importance','class','type','latitude','longitude','distance']


uc_location = (40.207422, -8.4260033)
cache = []
locations = geocode(place,    # search in Portugal
                    featuretype='settlement',
                    exactly_one=False, 
                    namedetails=True, addressdetails=True,
                    country_codes='pt')

if locations is None:       # if not in Brazil
    locations = geocode(place, 
                    featuretype='settlement',
                    exactly_one=False, 
                    namedetails=True, addressdetails=True,
                    country_codes='br')
if locations is None:      #if not in Portuguese speaking countries
    locations = geocode(place, 
                    featuretype='settlement',
                    exactly_one=False, 
                    namedetails=True, addressdetails=True,
                    country_codes=plp)
if locations is None:     # if not in other countries
    locations = geocode(place, 
                    featuretype='settlement',
                    exactly_one=False, 
                    namedetails=True, addressdetails=True,
                    country_codes=other_countries)

if locations is None:
    print("Not found:",place)
else:
    found = False
    for location in [loc for loc in locations]:
            
            if location.raw['class'] in ['place', 'boundary']:
                osm_id = location.raw.get('osm_id','*noif*')
                address_details = location.raw['address']
                osm_class = location.raw['class']
                osm_type = location.raw['type']
                lat = location.latitude
                long = location.longitude
                distance_in_km = geodesic(uc_location, (lat,long)).km
                country = address_details.get('country','')
                
                city = address_details.get('city',None)  # urban    
                town = address_details.get('town',None)  # rural
                municipality = address_details.get('municipality',None)  # rural

                city_name = city if city is not None \
                            else town if town is not None \
                            else municipality if municipality is not None \
                            else None
                if city_name is None:
                    continue
                address_details.pop('postcode',None)
                address_details.pop('country_code',None)

                address_details.pop('ISO3166-2-lvl4',None)
                address_details.pop('ISO3166-2-lvl3',None)

                osm_address = ", ".join(list(address_details.values()))  
                osm_importance = location.raw['importance']
                found = True
                cache = cache + [(place,'osm',osm_id, osm_address, city_name, country, osm_importance,osm_class,osm_type,lat,long,distance_in_km)]
            else:
                print("Ignoring",location.address,location.raw['class'])

    if found:        
        cache_df = pd.DataFrame(cache,columns=places_cache_cols)
        # sort by descending importance and increasing distance. First row should be the best
        cache_df = cache_df.sort_values(['importance','distance'], ascending=[False,True]).iloc[:1]
        print(f"Found: {place} ({cache_df['address'].iloc[0]})")    
    else:
        print("Not found:",place)       



Found: Porto (Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória, Porto, Porto, PT-13, Portugal)


In [10]:
cache_df

,naturalidade,geocoder,id,address,city,country,importance,class,type,latitude,longitude,distance
0,Porto,osm,3372453,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",Porto,Portugal,0.735184,boundary,administrative,41.149451,-8.610788,105.770156


In [11]:
places = naturalidades[naturalidades['count']>=5].sort_index().index
len(places)
places[:100]

Index(['', 'A de Barros', 'Abiul', 'Abiúl', 'Abragão', 'Abrantes',
       'Abrunheira', 'Abrunhosa', 'Adaúfe', 'Aguada de Baixo',
       'Aguada de Cima', 'Aguda', 'Aguiar', 'Aguiar da Beira', 'Aguieira',
       'Aguim', 'Alandroal', 'Albergaria', 'Albergaria a Velha',
       'Albergaria-a-Velha', 'Albufeira', 'Alcafache', 'Alcafozes', 'Alcaide',
       'Alcains', 'Alcanede', 'Alcanena', 'Alcangosta', 'Alcantarilha',
       'Alcaíde', 'Alcobaça', 'Alcochete', 'Alcoentre', 'Alcofra',
       'Alcongosta', 'Alcouce', 'Alcoutim', 'Alcácer', 'Alcácer do Sal',
       'Alcáçovas', 'Alcântara, Maranhão', 'Alcântara, Maranhão, Brasil',
       'Aldeia Galega', 'Aldeia Gavinha', 'Aldeia Nova',
       'Aldeia Nova das Donas', 'Aldeia Nova do Cabo', 'Aldeia da Cruz',
       'Aldeia da Cruz, Ourém', 'Aldeia da Mata', 'Aldeia da Ponte',
       'Aldeia das Dez', 'Aldeia de João Pires', 'Aldeia de Nogueira',
       'Aldeia do Mato', 'Alenquer', 'Alfandega da Fé', 'Alfarelos',
       'Alfeizerão', 'Alfe

### Improve

* Try to remove "orago" from name when not found. Orago in São|Santa+word+da|de|do+Place


### Misses hard to understand

* "Lagoa, Ilha de São Miguel, Açores"



### Mistakes
* Secarias,osm,280149141,"Secarias, Silveira, Silveira, Torres Vedras, Lisboa, Portugal",Silveira,Portugal,0.36,place,hamlet,39.1106425,-9.3582676,145.70092450095862 
    * should be https://pt.wikipedia.org/wiki/Secarias
    * better search first on wikidata by type= freguesia de Portugal

```sparql
   SELECT DISTINCT ?rank ?item ?itemLabel ?iof ?iofLabel ?inside ?insideLabel ?coordinates ?countryLabel WHERE 

   { SERVICE wikibase:mwapi 
              {bd:serviceParam wikibase:endpoint "pt.wikipedia.org";
                      wikibase:api "Generator";
                      mwapi:generator "search";
                      mwapi:gsrsearch "Secarias";
                      mwapi:gsrlimit "max".
               
      ?item wikibase:apiOutputItem mwapi:item .
    }
      
      ?item p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q1131296.
      ?item p:P31/ps:P31 ?iof.
    
      ?item p:P131/ps:P131 ?inside.   
    
      ?inside p:P31 ?statement1.
      ?statement1 (ps:P31/(wdt:P279*)) wd:Q13217644.
    
      ?item p:P625/ps:P625 ?coordinates.
      ?item p:P17/ps:P17 ?country.
    
   {SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }}
     }
  LIMIT 300
  ```


In [13]:
from os.path import exists
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

nplace = {'Roma':'Roma, Itália',
          'Santa Eulália de Barrosas':'Santa Eulália, Vizela',
          'Santa Eulália da Palmeira':'Palmeira, Santo Tirso',
          'Ribeira de Litém':'Santiago de Litém',
          'Baía':'Bahia, Brasil',
          'Baía, Brasil':'Bahia, Brasil',
          'Baía Brasil':'Bahia, Brasil',
          'Baia, Brasil':'Bahia, Brasil',
          'Baia Brasil':'Bahia, Brasil',
          'Alcangosta':'Alcongosta',
          "São João de El-Rei, Minas Gerais, Brasil": "São João del-Rei, Minas Gerais, Brasil",
          "São Paio de Farinha Podre, hoje São Pedro de Alva":"São Pedro de Alva",
          "Cabeceira de Basto":"Cabeceiras de Basto",
          "Vendas dos Moinhos": "Venda dos Moinhos"
          # Inglaterra
          # Ilha de S. Tomé
          }

uc_location = (40.207422, -8.4260033)

cache_file = '../inferences/places/osm-places.csv'
places_cache_cols = ['naturalidade','geocoder','id','address','city','country','importance','class','type','latitude','longitude','distance']
if exists(cache_file):
    places_cache = pd.read_csv(cache_file)
else:
   places_cache = pd.DataFrame(columns=places_cache_cols)


not_found_file = '../inferences/places/osm_not_found.csv'
if exists(not_found_file):
    not_found_df = pd.read_csv(not_found_file)
    not_found = list(not_found_df['not_found'])
else:
    not_found = []


geolocator = Nominatim(user_agent="fauc1530-1919", timeout=2)
# abide to https://operations.osmfoundation.org/policies/nominatim/
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1,  error_wait_seconds=3, return_value_on_exception = None) 

cached = set(places_cache['naturalidade'])
other_countries = ['br','es','it','fr']



for place in places:

    if place in nplace.keys():
        place = nplace[place]
    if place in cached:
        print(place, "cached.")
        continue
    if place in not_found:
        print(place, "previously not found.")
        continue

    cache = []
    locations = geocode(place, 
                        featuretype='settlement',
                        exactly_one=False, 
                        namedetails=True, addressdetails=True,
                        country_codes='pt')
    
    if locations is None:
        locations = geocode(place, 
                        featuretype='settlement',
                        exactly_one=False, 
                        namedetails=True, addressdetails=True,
                        country_codes=other_countries)

    if locations is None:
        print("Not found:",place)
        not_found = not_found + [place]
    else:

        for location in [loc for loc in locations]:
            found = False
            if location.raw['class'] in ['place', 'boundary']:
                osm_id = location.raw.get('osm_id','*noif*')
                address_details = location.raw['address']
                osm_class = location.raw['class']
                osm_type = location.raw['type']
                lat = location.latitude
                long = location.longitude
                distance_in_km = geodesic(uc_location, (lat,long)).km
                country = address_details.get('country','')
                
                city = address_details.get('city',None)  # urban    
                town = address_details.get('town',None)  # rural
                municipality = address_details.get('municipality',None)  # rural

                city_name = city if city is not None \
                            else town if town is not None \
                            else municipality if municipality is not None \
                            else None
                if city_name is None:
                    continue
                address_details.pop('postcode',None)
                address_details.pop('country_code',None)
                address_details.pop('ISO3166-2-lvl4',None)
                address_details.pop('ISO3166-2-lvl3',None)
                osm_address = ", ".join(list(address_details.values()))  
                osm_importance = location.raw['importance']
                found = True
                cache = cache + [(place,'osm',osm_id, osm_address, city_name, country, osm_importance,osm_class,osm_type,lat,long,distance_in_km)]

            else:
                print("Ignoring",location.address,location.raw['class'])

        if found:        
            cache_df = pd.DataFrame(cache,columns=places_cache_cols)

            # sort by descending importance and increasing distance. First row should be the best
            cache_df = cache_df.sort_values(['importance','distance'], ascending=[False,True]).iloc[:1]
            places_cache = pd.concat([places_cache,cache_df],axis=0)
            print(f"Found: {place} ({cache_df['address'].iloc[0]})")    
        else:
            print("Not found:",place)
            not_found_df = pd.concat([not_found_df,[(place,place)]], axis=0)      





Not found: 
Found: A de Barros (A-de-Barros, Penso, Penso e Freixinho, Sernancelhe, Viseu, Portugal)
Found: Abiul (Abiul, Pombal, Leiria, Portugal)
Found: Abiúl (Abiul, Pombal, Leiria, Portugal)
Found: Abragão (Abragão, Penafiel, Porto, PT-13, Portugal)
Ignoring Abrantes, Santarém, 2200-397, Portugal natural
Found: Abrantes (Abrantes, Santarém, Portugal)
Found: Abrunheira (Abrunheira, Montemor-o-Velho, Coimbra, Portugal)
Found: Abrunhosa (Abrunhosa, São Miguel de Vila Boa, Sátão, Viseu, Portugal)
Found: Adaúfe (Adaúfe, Braga, Braga, Portugal)
Found: Aguada de Baixo (Aguada de Baixo, Águeda, Aveiro, Portugal)
Found: Aguada de Cima (Aguada de Cima, Águeda, Aveiro, Portugal)
Ignoring Aguda, Porto de Mós, Leiria, 2480-032, Portugal natural
Found: Aguda (Aguda, Figueiró dos Vinhos, Leiria, Portugal)
Found: Aguiar (Aguiar, Barcelos, Braga, Portugal)
Found: Aguiar da Beira (Aguiar da Beira, Guarda, Portugal)
Ignoring Aguieira, Nazaré, Leiria, 2450-138, Portugal natural
Ignoring Aguieira, Vian

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rio de Janeiro, Brasil',), **{'featuretype': 'settlement', 'exactly_one': False, 'namedetails': True, 'addressdetails': True, 'country_codes': 'pt'}).
Traceback (most recent call last):
  File "/Users/jrc/.pyenv/versions/3.10.3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/jrc/.pyenv/versions/3.10.3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/jrc/.pyenv/versions/3.10.3/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/Users/jrc/.pyenv/versions/3.10.3/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/Users/jrc/.pyenv/versions/3.10.3/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.

Ignoring Rio de Janeiro, Três Marias, Região Geográfica Imediata de Curvelo, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, Brasil waterway
Found: Rio de Janeiro, Brasil (Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, Brasil)
Found: Rio de Moinhos (Rio de Moinhos, Penafiel, Porto, PT-13, Portugal)
Found: Rio de Vide (Rio de Vide, Miranda do Corvo, Coimbra, Portugal)
Found: Riodades (Riodades, São João da Pesqueira, Viseu, Portugal)
Found: Rios Frios (Rios Frios, Antuzede e Vil de Matos, Coimbra, Coimbra, Portugal)
Found: Roios (Roios, Vila Flor, Bragança, Portugal)
Found: Rojão Grande (Rojão Grande, Ovoa e Vimieiro, Santa Comba Dão, Viseu, Portugal)
Found: Romariz (Romariz, Romariz, Aveiro, Portugal)
Found: Roriz (Roriz, Barcelos, Braga, Portugal)
Found: Rosmaninhal (Rosmaninhal, Idanha-a-Nova, Castelo Branco, Portug

In [14]:
places_cache.to_csv(cache_file,index=False)
not_found_pd = pd.DataFrame(not_found,columns=['not_found'])
not_found_file = '../inferences/places/osm_not_found.csv'
not_found_pd.to_csv(not_found_file,index=False)



In [16]:
not_found_pd.head()

,not_found
0,Santa Maria de Fregim
1,Silhada
2,Santiago de Custóias
3,Santa Marinha de Arcozelo
4,"Pedrogão, Crato"
